Required Packages:

gc-ims-tools: Custom package to handle and analyze GC-IMS data

Pillow: For image handling (replacement for PIL)

OpenCV: For image processing

tensorflow: Deep learning framework (model, training, saliency maps)

tf-keras-vis: For generating saliency maps (e.g., SmoothGrad, etc.)

scikit-learn: For KFold, PCA, classification report, confusion matrix

scikit-image: For peak detection

numpy: For numerical array and matrix operations

matplotlib: For plotting images and graphs

seaborn: For enhanced plotting (e.g., confusion matrix heatmap)

tqdm: For progress bars in loops

Install all packages using pip:

pip install gc-ims-tools tensorflow tf-keras-vis scikit-learn scikit-image matplotlib seaborn tqdm Pillow opencv-python

_______________________________________________________________________________________________________________________

The following script performs preprocessing of GC-IMS data for use in machine learning applications, such as CNN-based classification.
The workflow includes data import, optional resolution reduction through binning, alignment of chromatograms using the
Reactant Ion Peak (RIP), cropping of non-informative regions along both drift time and retention time axes,
and exporting the resulting chromatograms as 2D heatmap images.

In [ ]:
# Import the IMS package for processing GC-IMS data
import ims

# Load the dataset from the specified directory. The dataset contains .mea files,
ds = ims.Dataset.read_mea(
    r"Dataset directory",
    subfolders=True,
)
print("Data Import Completed")

# Optional: Apply binning with a factor of 2 (reduce computational cost for system with limited RAM)
ds_bin = ds.binning(2)
print("Binning complete")

# Align chromatograms based on the Reactant Ion Peak (RIP) using relative interpolation.
# This step is very memory intensive.
ds_bin_rip = ds.interp_riprel()
print("RIP complete")

# Crop the chromatograms to remove non-informative or noisy regions:
# For Example:
# - Drift time is restricted to the range of 1.05 to 2.5 ms relative to RIP.
# - Retention time is limited to the range of 50 to 900 seconds.
ds_cut = ds_bin_rip.cut_dt(1.05, 2.5).cut_rt(50, 900)
print("Cut complete")

# Export the preprocessed chromatograms as 2D heatmap images (Image elements must be removed in the next step)
ds_cut.export_plots(r"Output directory")
print("Preprocessing Complete")

The following script performs standardized cropping of GC-IMS chromatogram images generated in the previous preprocessing step.
The cropping operation is necessary to isolate the informative central region of each heatmap while eliminating
axes, legends, and other non-data elements introduced during plotting. The script preserves the original
folder structure while saving the cropped images to a new output directory, maintaining consistency for downstream
classification tasks.

In [ ]:
from PIL import Image
import os

# Define the folder where images are stored
input_folder = r"Output directory of preprocessed chromatogram images"
output_folder = r"Output directory for final heatmaps"

# Define the crop area (left, upper, right, lower)
# Adjust these coordinates as per desired crop area
crop_area = (227, 122, 1342, 1507)

# Traverse through the input folder and its subfolders
for root, dirs, files in os.walk(input_folder):
    for image_name in files:
        if image_name.endswith(".jpg"):
            # Construct the full path to the image
            image_path = os.path.join(root, image_name)

            # Open the image file
            img = Image.open(image_path)

            # Crop the image using the defined crop area
            cropped_img = img.crop(crop_area)

            # Create corresponding subdirectory structure in the output folder
            relative_path = os.path.relpath(root, input_folder)
            output_subfolder = os.path.join(output_folder, relative_path)
            os.makedirs(output_subfolder, exist_ok=True)

            # Save the cropped image to the corresponding output subfolder
            cropped_img.save(os.path.join(output_subfolder, image_name))

            print(f"Cropped image {os.path.join(relative_path, image_name)} saved.")

This script implements a domain-specific data augmentation technique for GC-IMS chromatographic images, where peak regions are detected and selectively shifted vertically to simulate retention time variability The augmentation preserves chemical relevance by identifying peak locations using local maxima detection, 
then applying controlled directional shifts within predefined regions. 

In [ ]:
import cv2
import numpy as np
from skimage.feature import peak_local_max
import random
import os
import shutil
from pathlib import Path


def augment(image_path: str, output_path: str, direction: str):

    # Parameters
    Min_Peak_Distance = 15
    Peak_Threshold = 0.4
    Peak_Box_Size = 30
    Max_Vertical_Shift = 8
    Peak_Shift_Fraction = 0.5

    img = cv2.imread(image_path)
    if img is None:
        print(f"Could not read image {image_path}.")
        return False

    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_normalized = img_gray.astype(np.float32) / 255.0

    peak_coords = peak_local_max(
        img_normalized, min_distance=Min_Peak_Distance, threshold_abs=Peak_Threshold
    )

    if len(peak_coords) == 0:
        cv2.imwrite(output_path, img)
        return True

    augmented_img = np.copy(img)

    num_to_shift = int(len(peak_coords) * Peak_Shift_Fraction)
    if num_to_shift == 0 and len(peak_coords) > 0:
        num_to_shift = 1

    indices_to_shift = np.random.choice(
        len(peak_coords), size=num_to_shift, replace=False
    )

    for i in indices_to_shift:
        y_peak, x_peak = peak_coords[i]

        if direction == "up":
            # Shift must be negative (and not zero)
            shift = random.randint(-Max_Vertical_Shift, -1)
        elif direction == "down":
            # Shift must be positive (and not zero)
            shift = random.randint(1, Max_Vertical_Shift)
        else:  # 'both'
            shift = random.randint(-Max_Vertical_Shift, Max_Vertical_Shift)

        if shift == 0:
            continue

        half_box = Peak_Box_Size // 2
        y_start, y_end = y_peak - half_box, y_peak + half_box
        x_start, x_end = x_peak - half_box, x_peak + half_box
        new_y_start, new_y_end = y_start + shift, y_end + shift

        if (
            y_start < 0
            or y_end > img.shape[0]
            or x_start < 0
            or x_end > img.shape[1]
            or new_y_start < 0
            or new_y_end > img.shape[0]
        ):
            continue

        # Make a copy of the peak before modifying the image.
        peak_roi = augmented_img[y_start:y_end, x_start:x_end].copy()

        # Padding
        if shift > 0:  # Moved DOWN
            pad_source_y = max(0, y_start - 1)
            pad_row = augmented_img[pad_source_y, x_start:x_end]
            for y in range(y_start, y_end):
                augmented_img[y, x_start:x_end] = pad_row
        else:  # Moved UP
            pad_source_y = min(img.shape[0] - 1, y_end)
            pad_row = augmented_img[pad_source_y, x_start:x_end]
            for y in range(y_start, y_end):
                augmented_img[y, x_start:x_end] = pad_row

        # Place the original peak into the new location.
        augmented_img[new_y_start:new_y_end, x_start:x_end] = peak_roi

    cv2.imwrite(output_path, augmented_img)
    return True


def process_dataset(
    input_dir: str, output_dir: str, num_augmentations_per_image: int, direction: str
):
    input_path = Path(input_dir)
    output_path = Path(output_dir)
    print(f"Input directory: {input_path}")
    print(f"Output directory: {output_path}")

    image_files = (
        list(input_path.glob("**/*.jpg"))
        + list(input_path.glob("**/*.png"))
        + list(input_path.glob("**/*.jpeg"))
    )
    if not image_files:
        print("Error: No images found in the input directory.")
        return

    total_images = len(image_files)
    print(f"Found {total_images} images to process.")

    for i, image_file in enumerate(image_files):
        relative_path = image_file.relative_to(input_path)
        output_original_path = output_path / relative_path

        output_original_path.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy(image_file, output_original_path)

        for n in range(num_augmentations_per_image):
            aug_stem = output_original_path.stem + f"_aug_{n + 1}"
            aug_suffix = output_original_path.suffix
            output_augmented_path = output_original_path.with_name(
                f"{aug_stem}{aug_suffix}"
            )

            # Pass the direction parameter to the augmentation function
            augment(str(image_file), str(output_augmented_path), direction)

        print(
            f"({i+1}/{total_images}) Processed '{image_file.name}' -> saved original + {num_augmentations_per_image} augmented versions."
        )

    total_original = total_images
    total_augmented = total_images * num_augmentations_per_image
    grand_total = total_original + total_augmented
    print("\nProcessing complete.")
    print(
        f"Successfully created a new dataset with {grand_total} images in '{output_dir}'."
    )
    print(f"({total_original} original + {total_augmented} augmented)")


#
# Nomber of augmented versions for each original image
Num_Augmentations_Per_Image = 2

# Set the direction for the peak shift
Shift_Direction = "up"  # Options: 'up', 'down', 'both'

# Path to original dataset
Input_Directory = r"Output directory for final heatmaps"

# Path to augmented dataset
Output_Directory = r"Output directory for final heatmaps + Augmented images"

process_dataset(
    Input_Directory,
    Output_Directory,
    Num_Augmentations_Per_Image,
    Shift_Direction,
)

The following script implements a CNN-based classification pipeline using k-fold cross-validation to evaluate the model's generalization performance across GC-IMS heatmap images.
The pipeline includes model definition, image loading, cross-validated training, per-fold performance visualization, and statistical reporting of classification metrics with standard deviation.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict


# Define the CNN model architecture
def create_cnn_model(input_shape, num_classes):
    model = models.Sequential()
    model.add(
        layers.Conv2D(
            8, (3, 3), strides=(1, 1), padding="same", input_shape=input_shape
        )
    )
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling2D(2, 2))

    model.add(layers.Conv2D(32, (3, 3), strides=(1, 1), padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling2D(2, 2))

    model.add(layers.Flatten())
    model.add(layers.Dense(48, activation="relu", kernel_regularizer=l2(0.01)))
    model.add(layers.Dense(32, activation="relu", kernel_regularizer=l2(0.01)))
    model.add(layers.Dense(num_classes, activation="softmax"))

    return model


# Hyperparameters
input_shape = (248, 200, 3)
num_classes = 3
batch_size = 4
epochs = 50

# Create an ImageDataGenerator to rescale pixel values
datagen = ImageDataGenerator(rescale=1.0 / 255)

# Path to dataset directory containing class subfolders
data_dir = r"Directory for final heatmaps"

# Load all images and labels into memory
dataset = datagen.flow_from_directory(
    data_dir,
    target_size=(248, 200),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False,
)

# Store all image batches into arrays
X = []
Y = []
for i in range(len(dataset)):
    x_batch, y_batch = dataset[i]
    X.append(x_batch)
    Y.append(y_batch)
X = np.concatenate(X)
Y = np.concatenate(Y)

# Initialize K-Fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
acc_per_fold = []
loss_per_fold = []
histories = []

# Dictionary for storing precision, recall, and F1-score per class per fold
test_metrics_per_class = defaultdict(list)

# Begin k-fold training and evaluation
for fold, (train_idx, val_idx) in enumerate(kfold.split(X, Y)):
    print(f"Training fold {fold + 1}...")

    # Build and compile a new model instance for this fold
    model = create_cnn_model(input_shape, num_classes)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )

    X_train, X_val = X[train_idx], X[val_idx]
    Y_train, Y_val = Y[train_idx], Y[val_idx]

    # Train the model and store training history
    history = model.fit(
        X_train,
        Y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_val, Y_val),
        verbose=1,
    )
    histories.append(history)

    # Evaluate model on validation data
    scores = model.evaluate(X_val, Y_val)
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    print(
        f"Score for fold {fold + 1}: Loss = {scores[0]:.4f}; Accuracy = {scores[1] * 100:.2f}%"
    )

    # Plot accuracy and loss for this fold
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history["accuracy"], label="Train Accuracy")
    plt.plot(history.history["val_accuracy"], label="Val Accuracy", linestyle="--")
    plt.title(f"Fold {fold + 1} Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history["loss"], label="Train Loss")
    plt.plot(history.history["val_loss"], label="Val Loss", linestyle="--")
    plt.title(f"Fold {fold + 1} Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()

    plt.tight_layout()
    plt.savefig(f"accuracy_loss_fold_{fold + 1}.png")
    plt.close()

    # Save trained model for this fold
    model.save(f"cnn_model_fold_{fold + 1}.h5")
    print(f"Model for fold {fold + 1} saved.")

    # Generate and store classification metrics
    Y_val_pred = model.predict(X_val)
    y_true = np.argmax(Y_val, axis=1)
    y_pred = np.argmax(Y_val_pred, axis=1)
    report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)

    for cls in report:
        if cls in ["accuracy", "macro avg", "weighted avg"]:
            continue
        for metric in ["precision", "recall", "f1-score"]:
            test_metrics_per_class[f"{cls}_{metric}"].append(report[cls][metric])

# Plot average accuracy and loss across folds
average_train_accuracy = np.mean([h.history["accuracy"] for h in histories], axis=0)
average_val_accuracy = np.mean([h.history["val_accuracy"] for h in histories], axis=0)
average_loss = np.mean([h.history["loss"] for h in histories], axis=0)

plt.figure(figsize=(15, 6))
plt.subplot(1, 2, 1)
plt.plot(average_train_accuracy, label="Avg Train Accuracy", color="green")
plt.plot(average_val_accuracy, label="Avg Val Accuracy", linestyle="--", color="blue")
plt.title("Average Accuracy Across Folds")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(average_loss, label="Avg Loss", color="orange")
plt.title("Average Loss Across Folds")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.tight_layout()
plt.savefig("average_accuracy_loss_across_folds.png")
plt.close()

# Print overall performance summary
print("Average scores across all folds:")
print(f"> Accuracy: {np.mean(acc_per_fold):.2f}% (± {np.std(acc_per_fold):.2f}%)")
print(f"> Loss: {np.mean(loss_per_fold):.4f}")

# Display class-wise metrics with standard deviation
print("\nFinal Classification Report with Standard Deviation Across Folds:")
class_names = list(dataset.class_indices.keys())
for i, cls in enumerate(class_names):
    print(f"\nClass '{cls}' (Label {i}):")
    for metric in ["precision", "recall", "f1-score"]:
        values = test_metrics_per_class[f"{i}_{metric}"]
        mean_val = np.mean(values)
        std_val = np.std(values)
        print(f"  {metric}: {mean_val:.4f} ± {std_val:.4f}")

The following script implements a CNN-based classification pipeline for GC-IMS heatmap images with repeated random train-test splits to assess the model's performance robustness. 
The pipeline includes model definition, image loading, training across multiple random splits, per-run evaluation and visualization, and statistical reporting of classification metrics (precision, recall, F1-score) with mean and standard deviation across multiple runs.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns


# Define the CNN model architecture
def create_cnn_model(input_shape, num_classes):
    model = models.Sequential()
    model.add(
        layers.Conv2D(
            8, (3, 3), strides=(1, 1), padding="same", input_shape=input_shape
        )
    )
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling2D(2, 2))

    model.add(layers.Conv2D(32, (3, 3), strides=(1, 1), padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling2D(2, 2))

    model.add(layers.Flatten())
    model.add(layers.Dense(48, activation="relu", kernel_regularizer=l2(0.01)))
    model.add(layers.Dense(32, activation="relu", kernel_regularizer=l2(0.01)))
    model.add(layers.Dense(num_classes, activation="softmax"))

    return model


# Hyperparameters
input_shape = (248, 200, 3)
num_classes = 3
batch_size = 4
epochs = 50

# Create an ImageDataGenerator to rescale pixel values
datagen = ImageDataGenerator(rescale=1.0 / 255)

# Path to dataset directory containing class subfolders
data_dir = r"Directory for final heatmaps"

# Number of repeated random train/test runs
n_runs = 5
overall_accuracies = []
overall_losses = []
all_true_labels = []
all_pred_labels = []

# Lists to store per-class metrics across runs
precision_list = []
recall_list = []
f1_score_list = []

# Repeat training/testing over multiple random splits
for run in range(n_runs):

    # Load all images and labels into memory
    dataset = datagen.flow_from_directory(
        data_dir,
        target_size=(248, 200),
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=True,
    )

    # Store all image batches into arrays
    X = []
    Y = []
    for i in range(len(dataset)):
        x_batch, y_batch = dataset[i]
        X.append(x_batch)
        Y.append(y_batch)

    X = np.concatenate(X)
    Y = np.concatenate(Y)

    print(f"\nRun {run+1}/{n_runs} on random split:")

    # Create and compile the CNN model
    cnn_model = create_cnn_model(input_shape, num_classes)
    cnn_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )

    # Split data randomly into 80% train and 20% test
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size=0.2, shuffle=True
    )

    # Train the model and store training history
    history = cnn_model.fit(
        X_train,
        Y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_test, Y_test),
    )

    # Evaluate model on the test set
    test_scores = cnn_model.evaluate(X_test, Y_test)
    print(f"Test loss: {test_scores[0]}")
    print(f"Test accuracy: {test_scores[1]*100}%")

    # Save test accuracy and loss for this run
    overall_accuracies.append(test_scores[1])
    overall_losses.append(test_scores[0])

    # Predict on test set
    Y_pred = cnn_model.predict(X_test)
    Y_pred_classes = np.argmax(Y_pred, axis=1)
    Y_true_classes = np.argmax(Y_test, axis=1)

    # Store predictions and ground truths for cumulative confusion matrix
    all_true_labels.extend(Y_true_classes)
    all_pred_labels.extend(Y_pred_classes)

    # Generate and print classification report
    print(f"Classification Report for Run {run+1}:")
    report = classification_report(Y_true_classes, Y_pred_classes, output_dict=True)
    print(report)

    # Extract and store precision, recall, and F1-score per class
    precision_list.append([report[str(i)]["precision"] for i in range(num_classes)])
    recall_list.append([report[str(i)]["recall"] for i in range(num_classes)])
    f1_score_list.append([report[str(i)]["f1-score"] for i in range(num_classes)])

    # Confusion matrix for this run
    cm = confusion_matrix(Y_true_classes, Y_pred_classes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(
        cm,
        annot=True,
        fmt="d",
        cmap="Blues",
        xticklabels=dataset.class_indices.keys(),
        yticklabels=dataset.class_indices.keys(),
    )
    plt.title(f"Confusion Matrix for Run {run+1}")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.tight_layout()
    plt.savefig(f"confusion_matrix_run_{run+1}.png")
    plt.show()

    # Plot training and validation accuracy/loss
    plt.figure(figsize=(12, 6))

    # Accuracy plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history["accuracy"], label="Train Accuracy")
    plt.plot(
        history.history["val_accuracy"], label="Validation Accuracy", linestyle="--"
    )
    plt.title(f"Run {run+1} - Model Accuracy")
    plt.ylabel("Accuracy")
    plt.xlabel("Epochs")
    plt.legend(loc="lower right")

    # Loss plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history["loss"], label="Train Loss")
    plt.plot(history.history["val_loss"], label="Validation Loss", linestyle="--")
    plt.title(f"Run {run+1} - Model Loss")
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend(loc="upper right")

    plt.tight_layout()
    plt.savefig(f"accuracy_loss_plot_run_{run+1}.png")
    plt.show()

# Print mean accuracy and loss across runs
print("\nSummary of results from all runs:")
print(f"Average accuracy: {np.mean(overall_accuracies)*100:.2f}%")
print(f"Average loss: {np.mean(overall_losses):.4f}")

# Compute average and standard deviation of metrics
precision_mean = np.mean(precision_list, axis=0)
precision_std = np.std(precision_list, axis=0)
recall_mean = np.mean(recall_list, axis=0)
recall_std = np.std(recall_list, axis=0)
f1_score_mean = np.mean(f1_score_list, axis=0)
f1_score_std = np.std(f1_score_list, axis=0)

# Display final classification report with error bars
print("\nOverall Classification Report (Mean ± Standard Deviation):")
print("Class\tPrecision\tRecall\tF1-Score")
for i in range(num_classes):
    print(
        f"{i}\t{precision_mean[i]:.2f} ± {precision_std[i]:.2f}\t{recall_mean[i]:.2f} ± {recall_std[i]:.2f}\t{f1_score_mean[i]:.2f} ± {f1_score_std[i]:.2f}"
    )

# Create and save the final cumulative confusion matrix
overall_cm = confusion_matrix(all_true_labels, all_pred_labels)
plt.figure(figsize=(8, 6))
sns.heatmap(
    overall_cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=dataset.class_indices.keys(),
    yticklabels=dataset.class_indices.keys(),
)
plt.title("Overall Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.tight_layout()
plt.savefig("overall_confusion_matrix.png")
plt.show()

The following script performs inference using a pre-trained CNN model on GC-IMS heatmap images.
The pipeline includes model loading, preprocessing of test images using consistent normalization, generating predictions, mapping predicted indices to class labels, and saving the results to a CSV file for further analysis.

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

# Define image parameters and model path
image_size = (248, 200)  # Target size for image resizing (height, width)
input_shape = (248, 200, 3)  # Shape expected by the model (H, W, Channels)
batch_size = 4  # Must match the batch size used during training
model_path = r"Path to the trained CNN model"

# Load the trained model from file
model = tf.keras.models.load_model(model_path)

# Set the path to the test image directory
test_data_dir = r"Directory containing test images organized in subfolders"

# Define preprocessing pipeline consistent with training
datagen = ImageDataGenerator(rescale=1.0 / 255)

# Load test images using the data generator
test_generator = datagen.flow_from_directory(
    test_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode=None,  # No labels are used for prediction
    shuffle=False,  # Ensure filenames and predictions remain aligned
)

# Generate predictions on the test dataset
predictions = model.predict(test_generator, verbose=1)
predicted_classes = np.argmax(
    predictions, axis=1
)

# Map class indices to actual class labels
class_indices = (
    test_generator.class_indices
) 
idx_to_class = {v: k for k, v in class_indices.items()}
predicted_labels = [
    idx_to_class[i] for i in predicted_classes
] 

# Retrieve filenames corresponding to the predictions
filenames = test_generator.filenames

# Display predictions alongside corresponding filenames
for fname, pred_label in zip(filenames, predicted_labels):
    print(f"{fname} => {pred_label}")

# Optional: Save predictions and filenames to a CSV file
import pandas as pd

df = pd.DataFrame({"Filename": filenames, "Predicted Class": predicted_labels})
df.to_csv("predictions.csv", index=False)
print("Predictions saved to predictions.csv")

The following script generates class-specific saliency maps for GC-IMS heatmap images using a CNN model.
It loads a model and test dataset, computes SmoothGrad-enhanced saliency maps using tf-keras-vis, and visualizes them with a custom colormap. The resulting heatmaps are saved with appropriate scientific axes and labels, enabling interpretation of which regions most influence the model's classification decisions.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from tf_keras_vis.saliency import Saliency
from tf_keras_vis.utils.scores import CategoricalScore
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.ticker as ticker
from tqdm import tqdm

# Load the trained CNN model
model = tf.keras.models.load_model(r"Path to the trained CNN model")

# Set up data generator for normalization and image loading
datagen = ImageDataGenerator(rescale=1.0 / 255)
data_dir = r"Directory for final heatmaps"
target_size = (248, 200)

# Load dataset without shuffling to preserve order
val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=target_size,
    batch_size=1,
    class_mode="categorical",
    shuffle=False,  # Important: ensures filename-prediction alignment
)

# Create output directory for saving saliency maps
output_dir = r"Output directory for saving saliency maps"
os.makedirs(output_dir, exist_ok=True)

# Initialize Saliency from tf-keras-vis
saliency = Saliency(model)

# Generate saliency maps for all images in the dataset
for i in tqdm(range(len(val_generator)), desc="Generating saliency maps"):
    X_val, Y_val = val_generator[i]
    class_index = np.argmax(Y_val)
    filename = val_generator.filenames[i].replace("/", "_").replace("\\", "_")

    # Define score function for the true class
    score = CategoricalScore([class_index])

    # Compute SmoothGrad-enhanced saliency map
    saliency_map = saliency(score, X_val, smooth_samples=300, smooth_noise=0.05)[0]

    # Custom colormap: white (low saliency) to red (high saliency)
    white_to_red = LinearSegmentedColormap.from_list("white_red", ["white", "red"])

    # Apply cutoff threshold (optional)
    cutoff = 0.0
    saliency_map = np.maximum(saliency_map, cutoff)

    # Plot saliency map with scientific axis formatting
    fig, ax = plt.subplots(figsize=(4, 4), dpi=300)

    im = ax.imshow(
        saliency_map,
        cmap=white_to_red,
        vmin=cutoff,
        vmax=saliency_map.max(),
        extent=[1, 2.5, 50, 900],  # [dt_min, dt_max, rt_min, rt_max]
        origin="upper",
        aspect="auto",
    )

    # Axis labeling and tick formatting
    ax.set_xlabel("Drift Time")
    ax.set_ylabel("Retention Time")
    ax.xaxis.set_major_locator(ticker.MultipleLocator(0.5))
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(0.1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(100))
    ax.yaxis.set_minor_locator(ticker.MultipleLocator(20))
    ax.tick_params(which="major", length=6, labelsize=8)
    ax.tick_params(which="minor", length=3, labelsize=0)

    # Colorbar with label
    cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label("Saliency Intensity", fontsize=8)
    cbar.ax.tick_params(labelsize=6)

    # Save plot to output directory
    plt.tight_layout()
    plt.savefig(
        os.path.join(output_dir, f"saliency_{filename}.png"), bbox_inches="tight"
    )
    plt.close()

The following script computes class-specific saliency maps for GC-IMS heatmap images using a trained CNN model. Instead of visualizing the saliency maps directly, it flattens each map into a 1D feature vector and groups them according to their predicted class. The resulting feature vectors are saved as NumPy arrays (.npy) per class, forming a structured representation of salient regions for each sample.

Note:
These saved matrices are used in the next section to perform PCA and generate a "single representative image per class".

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from tf_keras_vis.saliency import Saliency
from tf_keras_vis.utils.scores import CategoricalScore
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib.colors import LinearSegmentedColormap
from tqdm import tqdm

# Load trained CNN model
model = tf.keras.models.load_model(r"Path to the trained CNN model")

# Set up data generator for normalization and image loading
datagen = ImageDataGenerator(rescale=1.0 / 255)
data_dir = r"Directory for final heatmaps"
target_size = (248, 200)

# Load dataset without shuffling to preserve order
val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=target_size,
    batch_size=1,
    class_mode="categorical",
    shuffle=False,
)

# Prepare output directories for Feature matrices
output_dir = r"Output directory for saving feature matrices"
feature_output_dir = os.path.join(output_dir, "Feature_Matrices")
os.makedirs(output_dir, exist_ok=True)
os.makedirs(feature_output_dir, exist_ok=True)

# Initialize Saliency from tf-keras-vis
saliency = Saliency(model)

# Custom colormap: white (low saliency) to red (high saliency)
white_to_red = LinearSegmentedColormap.from_list("white_red", ["white", "red"])

# Apply cutoff threshold (optional)
cutoff = 0.0

# Dictionary to collect flattened saliency vectors by class
feature_matrices = {}

# Generate saliency maps for all images in the dataset
for i in tqdm(range(len(val_generator)), desc="Generating saliency maps"):
    X_val, Y_val = val_generator[i]
    class_index = np.argmax(Y_val)
    filename = val_generator.filenames[i].replace("/", "_").replace("\\", "_")

    # Define score function for the current class
    score = CategoricalScore([class_index])

    # Compute SmoothGrad-enhanced saliency map
    saliency_map = saliency(score, X_val, smooth_samples=300, smooth_noise=0.05)[0]

    # Apply cutoff threshold
    saliency_map = np.maximum(saliency_map, cutoff)

    # Flatten saliency map to 1D vector
    feature_vector = saliency_map.flatten()

    # Collect feature vector under corresponding class
    if class_index not in feature_matrices:
        feature_matrices[class_index] = []
    feature_matrices[class_index].append(feature_vector)

# Save the feature matrices per class
for class_idx, vectors in feature_matrices.items():
    matrix = np.stack(vectors)  # shape: [num_samples, num_features]
    np.save(
        os.path.join(feature_output_dir, f"class_{class_idx}_saliency_matrix.npy"),
        matrix,
    )

The following script performs PCA on the class-specific saliency matrices generated earlier.
Each matrix contains flattened saliency maps for one class, where rows represent samples and columns represent pixel-wise features. PCA is applied to reduce dimensionality and extract the dominant patterns across all samples of a class.
The first principal component (PC1), also known as the loading vector, is reshaped back to the original image dimensions
and visualized as a heatmap. This representative image highlights the most influential and class-discriminative regions
across all saliency maps for that class.

Note:
These loading images serve as a "single representative visualization per class", revealing which regions of the GC-IMS
heatmaps are consistently important across all samples in that category.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from matplotlib.ticker import MultipleLocator
import matplotlib.ticker as ticker

# Directory containing saved saliency feature matrices (one .npy file per class)
feature_matrix_dir = r"Input directory for feature matrices"

# Directory to store PCA loading images (output)
pca_output_dir = r"Directory to store PCA loading images output"
os.makedirs(pca_output_dir, exist_ok=True)

# Original dimensions of the saliency maps
height, width = 248, 200

# Loop through all class-wise feature matrices
for filename in os.listdir(feature_matrix_dir):
    if filename.endswith(".npy") and filename.startswith("class_"):
        # Extract class index from filename
        class_idx = filename.split("_")[1]

        # Load feature matrix: shape = (num_samples, height * width)
        matrix = np.load(os.path.join(feature_matrix_dir, filename))

        # Perform PCA on the feature matrix
        pca = PCA()
        pca.fit(matrix)

        # Print explained variance ratio for reference (e.g., to verify PC1 contribution)
        print(pca.explained_variance_ratio_)

        # Extract the first principal component (loading vector)
        pc1 = pca.components_[0].reshape((height, width))  # Reshape to image format

        # Normalize PC1 to [-1, 1] for visualization
        max_abs = np.max(np.abs(pc1))
        pc1_normalized = pc1 / max_abs

        # Create figure for visualization
        fig, ax = plt.subplots(figsize=(4, 4), dpi=300)

        # Display the normalized PC1 as a heatmap using a blue-white-red colormap
        im = ax.imshow(
            pc1_normalized,
            cmap="bwr",
            vmin=-1,
            vmax=1,
            extent=[1, 2.5, 50, 900],  # Drift time (x) and Retention time (y) axes
            origin="upper",  # Maintain correct axis direction
            aspect="auto",
        )

        # Axis labels
        ax.set_xlabel("Drift time RIP relative (ms)")
        ax.set_ylabel("Retention Time (s)")

        # Set major and minor ticks for better interpretability
        ax.xaxis.set_major_locator(ticker.MultipleLocator(0.5))
        ax.xaxis.set_minor_locator(ticker.MultipleLocator(0.1))
        ax.yaxis.set_major_locator(ticker.MultipleLocator(100))
        ax.yaxis.set_minor_locator(ticker.MultipleLocator(20))

        # Tweak tick appearance
        ax.tick_params(which="major", length=6, labelsize=8)
        ax.tick_params(which="minor", length=3, labelsize=0)

        # Add colorbar to indicate saliency loading intensity
        cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
        cbar.set_label("Saliency Intensity", fontsize=8)
        cbar.ax.tick_params(labelsize=6)

        # Save the PC1 loading image for the class
        save_path = os.path.join(pca_output_dir, f"class_{class_idx}_pc1_loading.png")
        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
        plt.close()

        print(f"Saved: {save_path}")